In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model, preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
import time

In [2]:
df = pd.read_csv('df_for_model.csv')
df = df.drop('Unnamed: 0',axis=1)

In [3]:
SiteEnergyUse_discrete = pd.qcut(df['SiteEnergyUse(kBtu)'],3, labels = False)
GHGEmission_discrete = pd.qcut(df['GHGEmissions(MetricTonsCO2e)'],3, labels = False)
df.insert(11, 'SiteEnergyUse_discrete', SiteEnergyUse_discrete)
df.insert(12, 'GHGEmission_discrete', GHGEmission_discrete)

In [4]:
df['SiteEnergyUse(kBtu)_log'] = np.log1p(df['SiteEnergyUse(kBtu)'])
df['GHGEmissions(MetricTonsCO2e)_log'] = np.log1p(df['GHGEmissions(MetricTonsCO2e)'])

In [5]:
df=df.drop(df[df['PrimaryPropertyType']=='Hospital'].index)

In [48]:
df['PropertyGFAParking'].isna().sum()

0

In [6]:
df.columns

Index(['PrimaryPropertyType', 'Neighborhood', 'YearBuilt', 'NumberofBuildings',
       'NumberofFloors', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'ListOfAllPropertyUseTypes', 'LargestPropertyUseType',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType',
       'SiteEnergyUse_discrete', 'GHGEmission_discrete',
       'SecondLargestPropertyUseTypeGFA', 'ENERGYSTARScore',
       'SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)',
       'MedianEnergyUse', 'MedianGHGEmission', 'SiteEnergyUse(kBtu)_log',
       'GHGEmissions(MetricTonsCO2e)_log'],
      dtype='object')

In [7]:
data=df['Neighborhood'].values.reshape(-1,1)
enc = preprocessing.OneHotEncoder()
neighborhood_enc = enc.fit_transform(data).toarray()
neighborhood_enc.shape

(1800, 13)

In [8]:
data=df['PrimaryPropertyType'].values.reshape(-1,1)
enc = preprocessing.OneHotEncoder()
PropertyType_enc = enc.fit_transform(data).toarray()
PropertyType_enc.shape

(1800, 24)

In [9]:
X_cat = np.concatenate((neighborhood_enc,PropertyType_enc),axis=1)
X_cat.shape

(1800, 37)

In [10]:
#X_cat = np.concatenate((X_cat,df['SiteEnergyUse_discrete'].to_numpy().reshape(-1,1)),axis=1)

In [11]:
X_quant = df[['YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFABuilding(s)','PropertyGFAParking','LargestPropertyUseTypeGFA','SecondLargestPropertyUseTypeGFA','MedianEnergyUse','MedianGHGEmission']]
X_quant.values.shape

(1800, 9)

In [12]:
X_cat.shape

(1800, 37)

In [13]:
Y_energy_log = df['SiteEnergyUse(kBtu)_log']
Y_ghg_log = df['GHGEmissions(MetricTonsCO2e)_log']
Y_energy_bis = df['SiteEnergyUse(kBtu)']
Y_ghg_bis = df['GHGEmissions(MetricTonsCO2e)']

In [14]:
X_cat_train, X_cat_test, X_quant_train, X_quant_test, Y_energy_log_train, Y_energy_log_test, Y_ghg_log_train, Y_ghg_log_test, Y_energy_bis_train, Y_energy_bis_test, Y_ghg_bis_train, Y_ghg_bis_test = train_test_split(X_cat, X_quant, Y_energy_log, Y_ghg_log, Y_energy_bis, Y_ghg_bis, test_size=0.3, stratify = df['PrimaryPropertyType'], random_state = 1)

In [15]:
std = preprocessing.StandardScaler()
X_quant_train_scaled = std.fit_transform(X_quant_train)
X_quant_test_scaled = std.transform(X_quant_test)

In [16]:
X_train = np.concatenate((X_cat_train,X_quant_train_scaled),axis=1)
X_test = np.concatenate((X_cat_test,X_quant_test_scaled),axis=1)

In [17]:
# Régression linéaire sur cibles en log
lr_energy = linear_model.LinearRegression()
lr_energy.fit(X_train,Y_energy_log_train)

print('R² test sur variable log:', round(metrics.r2_score(Y_energy_log_test,lr_energy.predict(X_test)),3))
print('R² test sur variable origine:', round(metrics.r2_score(np.expm1(Y_energy_log_test),np.expm1(lr_energy.predict(X_test))),3))

print('R² train sur variable log:',round(metrics.r2_score(Y_energy_log_train,lr_energy.predict(X_train)),3))
print('R² train sur variable orig:',round(metrics.r2_score(np.expm1(Y_energy_log_train),np.expm1(lr_energy.predict(X_train))),3))

R² test sur variable log: 0.467
R² test sur variable origine: -0.171
R² train sur variable log: 0.534
R² train sur variable orig: -11.785


In [18]:
# Régression linéaire sur cibles non transformées
lr_energy_bis = linear_model.LinearRegression()
lr_energy_bis.fit(X_train,Y_energy_bis_train)

print('R² test :', round(metrics.r2_score(Y_energy_bis_test,lr_energy_bis.predict(X_test)),3))
print('R² train :',round(metrics.r2_score(Y_energy_bis_train,lr_energy_bis.predict(X_train)),3))

R² test : 0.387
R² train : 0.786


In [19]:
recap_energy = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for energy')
recap_ghg = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for GHG')

In [20]:
recap_energy_log = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for energy log')
recap_ghg_log = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for GHG log')

In [21]:
def perf(X_train,Y_train,X_test,Y_test,model,hyperparam):
    
    '''Fonction permettant la recherche des meilleurs hyperparamètres d'un modèle puis 
    l'entrainement de ce modèle avec ces meilleurs paramètres sur le jeu X_train, Y_train
    La fonction retourne les performances du modèle (R², RMSE, temps d'éxécution) et les meilleurs paramètres
    
    le paramètre 'hyperparam' de la fonction est un dictionnaire
    
    Liste des hyperparamètres selon certains modèles :
    - Régression linéaire : pas d'hyperparamètre
    - Régression Ridge : alpha
    - Regression Lasso : alpha
    - Elasticnet : alpha et l1_ratio
    - SVR : estimator__C, estimator__epsilon et estimator__kernel
    - Random Forest : 
    '''
        
    model_gs = GridSearchCV(estimator = model, param_grid=hyperparam, cv=5)      
        
    model_gs.fit(X_train, Y_train)
    start_time = time.time()
    score = round(metrics.r2_score(Y_test,model_gs.predict(X_test)),5)     
    RMSE = round(metrics.mean_squared_error(Y_test,model_gs.predict(X_test), squared = False),5)
    pred_time = round(time.time()-start_time, 6)
    best_param = model_gs.best_params_
    for k, v in best_param.items():
        if type(v)!= str :
            best_param[k] = round(v, 6)
   
    return score, RMSE, pred_time, best_param



def recap_table(X_train,Y_energy_train,Y_ghg_train,X_test,Y_energy_test,Y_ghg_test,model,hyperparam):
    score, RMSE, pred_time, best_param = perf(X_train,Y_energy_train,X_test,Y_energy_test,model,hyperparam)
    recap_energy.loc[str(model),:] = [score,RMSE,pred_time,best_param]
    
    score, RMSE, pred_time, best_param = perf(X_train,Y_ghg_train,X_test,Y_ghg_test,model,hyperparam)
    recap_ghg.loc[str(model),:] = [score,RMSE,pred_time,best_param]

In [22]:
def perf_log(X_train,Y_train,X_test,Y_test,model,hyperparam):
    
    '''Fonction permettant la recherche des meilleurs hyperparamètres d'un modèle puis 
    l'entrainement de ce modèle avec ces meilleurs paramètres sur le jeu X_train, Y_train
    La fonction retourne les performances du modèle (R², RMSE, temps d'éxécution) et les meilleurs paramètres
    
    le paramètre 'hyperparam' de la fonction est un dictionnaire
    
    Liste des hyperparamètres selon certains modèles :
    - Régression linéaire : pas d'hyperparamètre
    - Régression Ridge : alpha
    - Regression Lasso : alpha
    - Elasticnet : alpha et l1_ratio
    - SVR : estimator__C, estimator__epsilon et estimator__kernel
    - Random Forest : 
    '''
        
    model_gs = GridSearchCV(estimator = model, param_grid=hyperparam, cv=5)      
        
    model_gs.fit(X_train, Y_train)
    start_time = time.time()
    score = round(metrics.r2_score(np.expm1(Y_test),np.expm1(model_gs.predict(X_test))),5)     
    RMSE = round(metrics.mean_squared_error(np.expm1(Y_test),np.expm1(model_gs.predict(X_test)), squared = False),5)
    pred_time = round(time.time()-start_time, 6)
    best_param = model_gs.best_params_
    for k, v in best_param.items():
        if type(v)!= str :
            best_param[k] = round(v, 6)
   
    return score,RMSE, pred_time, best_param



def recap_table_log(X_train,Y_energy_train,Y_ghg_train,X_test,Y_energy_test,Y_ghg_test,model,hyperparam):
    score, RMSE, pred_time, best_param = perf_log(X_train,Y_energy_train,X_test,Y_energy_test,model,hyperparam)
    recap_energy_log.loc[str(model),:] = [score,RMSE,pred_time,best_param]
    
    score, RMSE, pred_time, best_param = perf_log(X_train,Y_ghg_train,X_test,Y_ghg_test,model,hyperparam)
    recap_ghg_log.loc[str(model),:] = [score,RMSE,pred_time,best_param]

In [23]:
LinearReg = linear_model.LinearRegression();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,LinearReg,{})

In [24]:
ridge = Ridge();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,ridge,{'alpha': np.logspace(-4,2,100)})

In [25]:
lasso = Lasso();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,lasso,{'alpha': np.logspace(-4,2,100)})

C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.832578203786783e+16, tolerance: 96520725261772.31
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0528620046467982e+17, tolerance: 117614868306021.38
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0551758543102882e+17, tolerance: 118656246327521.9
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py

In [26]:
svr = SVR(kernel = 'linear');
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,svr,{'C' : range(1,5),
                                       'epsilon' : np.linspace(0.1,1,10)})

In [27]:
svr = SVR();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,svr,{'C' : range(1,5),
                                        'epsilon' : np.linspace(0.1,1,10), 
                                       'kernel' : ['rbf','poly','sigmoid']})

In [28]:
rfr = RandomForestRegressor();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test, rfr, {'min_samples_split' : [2, 5, 10], 
                                             'min_samples_leaf' : [1, 2, 4], 
                                             'max_depth': range(5,16,1)})

In [29]:
gb = LGBMRegressor();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test, gb, {'n_estimators': [150],'max_depth':range(5,16), 
                                                                                                     'learning_rate' : [0.0001,0.001,0.01,0.1,1], 
                                                                                                     'boosting_type' : ['gbdt','goss', 'dart'], 'num_leaves':[30]})

In [30]:
recap_energy

,R²,RMSE,Prediction time,Best parameters
Model for energy,,,,
LinearRegression(),0.38735,9.42452e+06,0.003999,{}
Ridge(),0.49228,8.57964e+06,0.002,{'alpha': 100.0}
Lasso(),0.38615,9.43379e+06,0.001,{'alpha': 100.0}
SVR(kernel='linear'),-0.11372,1.2707e+07,0.079007,"{'C': 4, 'epsilon': 0.1}"
SVR(),-0.11388,1.27079e+07,0.061495,"{'C': 4, 'epsilon': 0.1, 'kernel': 'poly'}"
RandomForestRegressor(),0.66454,6.97394e+06,0.036003,"{'max_depth': 6, 'min_samples_leaf': 2, 'min_s..."
LGBMRegressor(),0.58631,7.74448e+06,0.011001,"{'boosting_type': 'goss', 'learning_rate': 0.0..."


In [31]:
recap_ghg

,R²,RMSE,Prediction time,Best parameters
Model for GHG,,,,
LinearRegression(),-0.07042,350.937,0.002003,{}
Ridge(),0.11334,319.396,0.001999,{'alpha': 100.0}
Lasso(),0.06036,328.801,0.002,{'alpha': 6.135907}
SVR(kernel='linear'),0.23732,296.225,0.061004,"{'C': 4, 'epsilon': 0.1}"
SVR(),0.06302,328.334,0.100006,"{'C': 4, 'epsilon': 0.3, 'kernel': 'sigmoid'}"
RandomForestRegressor(),0.54661,228.395,0.031507,"{'max_depth': 12, 'min_samples_leaf': 2, 'min_..."
LGBMRegressor(),0.43954,253.937,0.011001,"{'boosting_type': 'goss', 'learning_rate': 0.0..."


In [32]:
LinearReg = linear_model.LinearRegression();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,LinearReg,{})

In [33]:
ridge = Ridge();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,ridge,{'alpha': np.logspace(-4,2,100)})

In [34]:
lasso = Lasso();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,lasso,{'alpha': np.logspace(-4,2,100)})

C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.9914536495248285, tolerance: 0.17517760807167526
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.010392415162755, tolerance: 0.17517760807167526
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.031579952163156, tolerance: 0.17517760807167526
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: Co

In [35]:
svr = SVR(kernel = 'linear');
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,svr,{'C' : range(1,5),
                                       'epsilon' : np.linspace(0.1,1,10)})

In [36]:
svr = SVR();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,svr,{'C' : range(1,5),
                                        'epsilon' : np.linspace(0.1,1,10), 
                                       'kernel' : ['rbf','poly','sigmoid']})

In [37]:
rfr = RandomForestRegressor();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test, rfr, {'min_samples_split' : [2, 5, 10], 
                                             'min_samples_leaf' : [1, 2, 4], 
                                             'max_depth': range(5,16,1)})

In [105]:
gb = LGBMRegressor();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test, gb, {'n_estimators': [300],'max_depth': [2], 
                                                                                                     'learning_rate' : [0.07], 
                                                                                                     'boosting_type' : ['gbdt','goss', 'dart']})

In [106]:
recap_energy_log

,R²,RMSE,Prediction time,Best parameters
Model for energy log,,,,
LinearRegression(),-0.17148,1.30323e+07,0.003001,{}
Ridge(),0.07045,1.16089e+07,0.002,{'alpha': 100.0}
Lasso(),0.16285,1.10168e+07,0.002001,{'alpha': 0.162975}
SVR(kernel='linear'),0.06562,1.16391e+07,0.021002,"{'C': 4, 'epsilon': 0.9}"
SVR(),0.48461,8.64417e+06,0.084009,"{'C': 4, 'epsilon': 0.2, 'kernel': 'rbf'}"
RandomForestRegressor(),0.62846,7.33937e+06,0.046004,"{'max_depth': 15, 'min_samples_leaf': 2, 'min_..."
LGBMRegressor(),0.67204,6.89545e+06,0.010001,"{'boosting_type': 'gbdt', 'learning_rate': 0.0..."


In [107]:
recap_ghg_log

,R²,RMSE,Prediction time,Best parameters
Model for GHG log,,,,
LinearRegression(),-0.00299,339.704,0.002002,{}
Ridge(),0.03597,333.04,0.002001,{'alpha': 100.0}
Lasso(),0.05227,330.213,0.001997,{'alpha': 0.162975}
SVR(kernel='linear'),-0.01962,342.508,0.027003,"{'C': 4, 'epsilon': 1.0}"
SVR(),0.21259,300.99,0.046004,"{'C': 3, 'epsilon': 0.6, 'kernel': 'rbf'}"
RandomForestRegressor(),0.46406,248.32,0.052005,"{'max_depth': 14, 'min_samples_leaf': 4, 'min_..."
LGBMRegressor(),0.54825,227.981,0.010002,"{'boosting_type': 'gbdt', 'learning_rate': 0.0..."


In [100]:
recap_energy_log.iloc[-1,-1]

{'boosting_type': 'gbdt',
 'learning_rate': 0.07,
 'max_depth': 6,
 'n_estimators': 125,
 'num_leaves': 30}